## 패션 이미지 데이터셋 전처리

In [1]:
import pandas as pd

path = 'data/meta/'
train = pd.read_json(path + 'train_no_dup.json')
valid = pd.read_json(path + 'valid_no_dup.json')
test = pd.read_json(path + 'test_no_dup.json')
blank = pd.read_json(path + 'fill_in_blank_test.json')

## Data Load Example

In [2]:
import pandas as pd
from importlib import reload
import sampling  # 이미 import된 모듈
reload(sampling)  # 모듈 다시 로드
from sampling import DataSampler 
import os

base_dir = os.getcwd()
data_dir = os.path.join(base_dir, 'data')
meta_dir = os.path.join(data_dir, 'meta')
image_dir = os.path.join(data_dir, 'images')
sampler = DataSampler(data_path = meta_dir, k=150, test_sampling_ratio=1)
concat_df, question_data = sampler.sample_data()


In [3]:
concat_df

,name,items,desc,set_id,type
0,Casual Eleanor.,"[{'index': 1, 'name': 'doublju women's crew ne...",A fashion look from August 2016 by haley-bob1 ...,205211881,train
1,Who wears short short !?,"[{'index': 1, 'name': 'abercrombie fitch appli...",A fashion look from July 2016 by blancacalleja...,203655321,train
2,Date Night,"[{'index': 1, 'name': 'dip dye raglan blue whi...",A fashion look from February 2017 by hattie4pa...,216829898,train
3,TOP,"[{'index': 1, 'name': 'forever new lauren embe...",A fashion look from December 2013 featuring sh...,107222824,train
4,interview outfit,"[{'index': 1, 'name': 'wool sweater', 'price':...",A fashion look from November 2016 by smartbuyg...,211086877,train
...,...,...,...,...,...
7382,Sheinside.com,"[{'index': 1, 'name': 'black sleeveless hollow...",#Sheinside,122358035,test
7383,Inspired by street style/ the new queen,"[{'index': 1, 'name': 'j.crew jeweled chandeli...",A fashion look from November 2014 featuring pi...,141687053,test
7384,*edgy chic*,"[{'index': 1, 'name': 'crop pullover', 'price'...",A fashion look from February 2014 featuring cr...,113788101,test
7385,Cute White Laser Loose Top Set: Contest Entry,"[{'index': 1, 'name': 'shein sheinside loose f...",A fashion look from June 2016 by krystinekerch...,199807808,test


In [8]:
concat_df['type'].value_counts()

type
train    4363
test     2283
valid     741
Name: count, dtype: int64

In [9]:
question_data

,question,answers,blank_position,set_id
0,"[119704139_1, 119704139_3, 119704139_4, 119704...","[119704139_2, 148511719_1, 206598043_4, 106043...",2,119704139
1,"[119314458_1, 119314458_2, 119314458_3, 119314...","[119314458_6, 209157393_8, 113308426_5, 119813...",6,119314458
2,"[147361785_1, 147361785_2, 147361785_3, 147361...","[147361785_4, 112144897_4, 102654372_1, 196192...",4,147361785
3,"[148015877_1, 148015877_2, 148015877_3]","[148015877_4, 152955044_4, 183769156_2, 186971...",4,148015877
5,"[163560873_1, 163560873_2, 163560873_3, 163560...","[163560873_4, 211633718_2, 176899499_5, 100675...",4,163560873
...,...,...,...,...
3071,"[203949517_1, 203949517_2, 203949517_3, 203949...","[203949517_5, 216610008_6, 129207446_7, 111531...",5,203949517
3072,"[196280291_2, 196280291_3, 196280291_4, 196280...","[196280291_1, 118801622_1, 214377821_7, 216835...",1,196280291
3073,"[216061834_1, 216061834_2, 216061834_4, 216061...","[216061834_3, 186088042_3, 139790914_1, 204133...",3,216061834
3074,"[195200115_1, 195200115_2, 195200115_3, 195200...","[195200115_5, 215249643_4, 146824478_6, 120556...",5,195200115


In [5]:
print(image_dir)

c:\Users\jsj91\.vscode\AI_MMR\data\images


In [64]:
concat_df[concat_df['type']=='test']

,name,items,desc,set_id,type
1027,Street Dance,"[{'index': 1, 'name': 'dolce gabbana silk shir...",A fashion look from April 2014 featuring cream...,119704139,test
1028,I Will Be Your Sunset If You'll Be My Silhouette!,"[{'index': 1, 'name': 'irregular lapel blue tr...",A fashion look from April 2014 by sherri-2loco...,119314458,test
1029,Edgy Urbanist,"[{'index': 1, 'name': 'women', 'price': 95.0, ...",2016/03/05\nHappy Weekend to everyone..have pl...,192637078,test
1030,My favorite bag,"[{'index': 1, 'name': 'derek lam draped cady t...",A fashion look from June 2015 by bliznec featu...,163560873,test
1031,Yoins - 24/08/2016 - 085,"[{'index': 1, 'name': 'yoins sequin hollow lon...",YOINS - Your Inspiration:\nshop here :http://y...,206598043,test
...,...,...,...,...,...
2862,Casual|,"[{'index': 1, 'name': 'hollister easy tee', 'p...",A fashion look from July 2016 by babylaci feat...,203949517,test
2863,Casual,"[{'index': 1, 'name': 'round neck short sleeve...",A fashion look from April 2016 by beebeely-loo...,196280291,test
2864,Winter White,"[{'index': 1, 'name': 'stella mccartney oversi...",A fashion look from February 2017 by adduncan ...,216061834,test
2865,Hot Ticket: Rihanna Concert,"[{'index': 1, 'name': 'women sexy hollow flora...",A fashion look from April 2016 by svijetlana f...,195200115,test


In [ ]:
from importlib import reload
import multimodal_data 
reload(multimodal_data)
from multimodal_data import MultiModalData

train_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='train')
valid_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, mode='valid')
test_dataset = MultiModalData(concat_df, sampler.category_df, image_dir, question = question_data, mode='test')

In [5]:
len(test_dataset)

2283

In [8]:
test_dataset[0]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [10]:
test_dataset[10]['question']

[{'question_ids': ['100445477_1',
   '100445477_3',
   '100445477_4',
   '100445477_5',
   '100445477_6',
   '100445477_7',
   '100445477_8'],
  'answer': ['100445477_2', '213956506_2', '190058983_5', '134261002_3'],
  'blank_position': 2}]

In [19]:
valid_dataset[10].keys()

dict_keys(['texts', 'text_gt', 'prices', 'likes', 'images', 'image_gt', 'set_id', 'valid_idx', 'gt_idx'])

## Dataloader

In [16]:
# from torch.utils.data import Dataset, DataLoader
# import torch
# import clip
# from PIL import Image

# def custom_collate_fn(batch, test=False):
#     texts = [item["texts"] for item in batch]
#     price = [item["price"] for item in batch]         
#     likes = [item["likes"] for item in batch]         
#     images = [item["images"] for item in batch]        
#     set_id = [item["set_id"] for item in batch]       
       
#     if test:
#         question = [item["question"] for item in batch] 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id,
#             "question": question,
#         }
#     else: 
#         return {
#             "texts": texts,
#             "price": price,
#             "likes": likes,
#             "images": images,
#             "set_id": set_id
#         }
    
# valid_loader = DataLoader(
#     valid_dataset, 
#     batch_size=4, 
#     shuffle=True, 
#     collate_fn=lambda batch: custom_collate_fn(batch, test=False)
# )

In [17]:
for batch in valid_loader:
    print(batch)

KeyError: 'price'